<a href="https://colab.research.google.com/github/yugokatsuki/EDA/blob/master/start_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()
    
drive_service = build('drive', 'v3')
results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
print(results)
print(kaggle_api_key)

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)


{'files': [{'id': '1VPZvpe5aOfNJNuKxlnOiSN0DTxD6JUIg'}]}
[{'id': '1VPZvpe5aOfNJNuKxlnOiSN0DTxD6JUIg'}]
Download 100%.


In [2]:
  !pip install kaggle


In [3]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         89           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        187           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        286           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2184           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      17170            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       4647           False  
connectx

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 9.04MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.21MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 45.3MB/s]


In [71]:
import numpy as np
import pandas as pd
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import time

def parse(df):
    # Nan を -1 に置換
    df["Age"] = df["Age"].fillna(-1)
    df["Embarked"] = df["Embarked"].fillna(-1)
    # 文字列を数値に変換
    df["Sex"][df["Sex"] == "male"] = 0
    df["Sex"][df["Sex"] == "female"] = 1
    df["Embarked"][df["Embarked"] == "S" ] = 0
    df["Embarked"][df["Embarked"] == "C" ] = 1
    df["Embarked"][df["Embarked"] == "Q"] = 2
    return df

def split_val(x,y,rate,seed=None):
  print(f"rate = {rate}")
  print("rate = {}".format(rate))
  
  if seed is not None:
    np.random.seed(seed)
  N = x.shape[0]
  perm = np.random.permutation(N)
  valnum = int(rate * N)
  index1 = perm[:(-valnum)]
  index2 = perm[(-valnum):]
  return x[index1],y[index1],x[index2],y[index2]

def load_data():
    train_csv = pd.read_csv('train.csv')
    #before inputting the data, extract the value 
    train_csv = parse(train_csv)
    train_x = train_csv[['Pclass', 'Sex', 'Fare','SibSp', 'Parch', 'Age', 'Embarked']].values
    train_y = train_csv['Survived'].values
    print(train_x.shape)
    return split_val(train_x,train_y,0.1)


In [73]:
def create_model(input_placeholder,u_dim,layer_num,y_dim,training):
    h = input_placeholder
    for i in range(layer_num):
        h = tf.layers.dense(inputs=h, units=u_dim,activation=tf.nn.relu)
        h = tf.layers.batch_normalization(h,training=training)  
    h = tf.layers.dense(inputs=h, units=y_dim)
    return h

np.random.seed(0)
tf.reset_default_graph()
#from tensorflow.keras import backend

y_dim     = 2
# hyperparameters
u_dim     = 100
layer_num = 2
epoch     = 1000
log_freq  = 100
batchsize = 200
lr        = 0.001

# data load
train_x,train_y,val_x,val_y = load_data()
N  = train_y.shape[0]
Nv = val_y.shape[0]
print('train num: {}, val num: {}'.format(N,Nv))

# calculation graph
x = tf.placeholder(tf.float32, [None, train_x.shape[1]],"input")
y = tf.placeholder(tf.int32, [None])

with tf.variable_scope("model"):
  train_z = create_model(x,u_dim,layer_num,y_dim,training=True)
with tf.variable_scope("model", reuse=True):
  z       = create_model(x,u_dim,layer_num,y_dim,training=False)

cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=train_z)

extra_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_ops):
  train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

pred_y = tf.cast(tf.argmax(z, 1), tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred_y, y), tf.float32))

# main loop
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
T = time.time() 
for ep in range(1,epoch+1) :
  perm=np.random.permutation(N)
  for i in range(0,N,batchsize):
    batch_xs=train_x[perm[i:i+batchsize]]
    batch_ys=train_y[perm[i:i+batchsize]]
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
  # monitor
  if ep%log_freq ==0 :      
    train_loss, train_acc = sess.run([cross_entropy, accuracy], feed_dict={x: train_x, y: train_y})
    val_loss, val_acc     = sess.run([cross_entropy, accuracy], feed_dict={x: val_x, y: val_y})
    epochT = time.time()-T
    print('Epoch: %d, Time :%.4f (s), train_loss: %f,  train_acc: %f, val_loss: %f,  val_acc: %f' % (ep, epochT, train_loss, train_acc, val_loss, val_acc))
    T = time.time()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_l

(891, 7)
rate = 0.1
rate = 0.1
train num: 802, val num: 89


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch: 100, Time :1.2418 (s), train_loss: 0.411638,  train_acc: 0.814214, val_loss: 0.441899,  val_acc: 0.775281
Epoch: 200, Time :1.2151 (s), train_loss: 0.425965,  train_acc: 0.770574, val_loss: 0.455679,  val_acc: 0.820225
Epoch: 300, Time :1.1913 (s), train_loss: 0.396882,  train_acc: 0.809227, val_loss: 0.438809,  val_acc: 0.752809
Epoch: 400, Time :1.1799 (s), train_loss: 0.415363,  train_acc: 0.824190, val_loss: 0.445120,  val_acc: 0.741573
Epoch: 500, Time :1.1903 (s), train_loss: 0.396813,  train_acc: 0.831671, val_loss: 0.447367,  val_acc: 0.786517
Epoch: 600, Time :1.2345 (s), train_loss: 0.401921,  train_acc: 0.831671, val_loss: 0.508341,  val_acc: 0.741573
Epoch: 700, Time :1.1779 (s), train_loss: 0.394775,  train_acc: 0.745636, val_loss: 0.537627,  val_acc: 0.741573
Epoch: 800, Time :1.2202 (s), train_loss: 0.374067,  train_acc: 0.837905, val_loss: 0.442154,  val_acc: 0.764045
Epoch: 900, Time :1.1858 (s), train_loss: 0.363301,  train_acc: 0.837905, val_loss: 0.433287,  v

In [74]:
# data
test_csv = pd.read_csv('test.csv')
test_csv = parse(test_csv)
test_x = test_csv[['Pclass', 'Sex', 'Fare','SibSp', 'Parch', 'Age', 'Embarked']].values
# predict
prediction = sess.run(pred_y, feed_dict={x: test_x})
# parse
PassengerId = np.array(test_csv["PassengerId"]).astype(int)
my_solution = pd.DataFrame(prediction, PassengerId, columns = ["Survived"])
# save
my_solution.to_csv("result.csv", index_label = ["PassengerId"])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_l

# 新しいセクション

In [75]:
!kaggle competitions submit -c titanic -f result.csv -m 'first submit'

100% 2.77k/2.77k [00:00<00:00, 12.8kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster